In [ ]:
# Setup
import jinja2
import numpy as np
import os
from OptStab import beta

OutDir = os.path.join(os.path.pardir,os.path.pardir,'data','results')


def FormatNumber(x,precision=3):
    if type(x) == str:
        return x
    elif type(x) == np.ndarray:
        return np.array([FormatNumber(xi) for xi in x])
    else:       
        formatstring = '{x:.' + str(precision) + 'f}'
        return formatstring.format(x = x)
        


env = jinja2.Environment(
        "%<", ">%",
        "<<", ">>",
        "[§", "§]",
        loader=jinja2.FileSystemLoader(".")
        )


RowTemplate =   '(<<rownum>>) & <<rowname>>   & <<b>> & <<tau>> & <<std_dWdx>> & <<std_dxdb>> & <<cov_dWdx_dxdb>> \\\\'

template = env.from_string(RowTemplate)

def RepRate(b,tau):
    return 2 * b **(1.0/(1-tau)) -1

def GetRow(Version, rownum,rowname,Shocks=True):
    
    with np.load(os.path.join(OutDir,'SearchResults_v'+str(Version)+'.npz')) as X:
        if Shocks:
            b = X['OptCyc'][0]
            tau = X['OptCyc'][1]
        else:
            b = X['OptSS'][0]
            tau = X['OptSS'][1]

    if Shocks:
        with np.load(os.path.join(OutDir,'Unpack_v'+str(Version)+'_param_b_mode_point.npz')) as X:
            std_dWdx = X['std_dWdx'][0] *(1-beta)
            std_dxdb = X['std_dxdparam'][0] *(1-beta)
#             cov_dWdx_dxdb = X['Cov'][0].sum() *(1-beta)
            cov_dWdx_dxdb = X['Cov'][0,2:].sum() *(1-beta)
    else:
        std_dWdx = std_dxdb = cov_dWdx_dxdb = '--'

    
    return template.render(rownum = rownum,
                    rowname = rowname,
                    b = FormatNumber(b,3),
                    tau = FormatNumber(tau,3),
                    std_dWdx = FormatNumber(std_dWdx,3),
                    std_dxdb = FormatNumber(std_dxdb,3),
                    cov_dWdx_dxdb = FormatNumber(cov_dWdx_dxdb,3) )
                
def GetRowFlex(Version, rownum,rowname):
    
    with np.load(os.path.join(OutDir,'SearchResults_v'+str(Version)+'.npz')) as X:
            b = X['OptCyc'][0]
            tau = X['OptCyc'][1]
   
    
    return template.render(rownum = rownum,
                    rowname = rowname,
                    b = FormatNumber(b,3),
                    tau = FormatNumber(tau,3),
                    std_dWdx = '--',
                    std_dxdb = 'See note.',
                    cov_dWdx_dxdb = '--' )
                



In [ ]:
fname = os.path.join(OutDir,'MacroStabTable.tex')
f = open(fname, "w")

f.write(os.linesep)
f.write(GetRow(1,'i','No aggr. shocks',Shocks=False))
f.write(os.linesep)
f.write(GetRow(1,'ii','Baseline'))
f.write(os.linesep)
f.write(GetRowFlex(8,'iii','Flexible prices'))
f.write(os.linesep)
f.write(GetRow(3,'iv','Aggressive monetary policy'))
f.write(os.linesep)
f.write(GetRow(22,'v','Smaller mon. shock'))
f.write(os.linesep)
f.write(GetRow(12,'vi','No $Q^u$'))
f.write(os.linesep)
# f.write(GetRow(13,'vi','No $Q^\epsilon$'))
# f.write(os.linesep)
f.write(GetRow(16,'vii','No skill risk'))
f.write(os.linesep)
f.write(GetRow(31,'viii','More cyclical skill risk'))
f.write(os.linesep)
f.write(GetRow(10,'ix','Acyclical G'))
f.write(os.linesep)
f.write(GetRow(32,'x','Higher G/Y ratio, w/o shocks',Shocks=False))
f.write(os.linesep)
f.write(GetRow(32,'x','Acyclical G, Higher G/Y ratio'))
f.write(os.linesep)
f.write(GetRow(30,'xi','More elastic labor supply, w/o shocks',Shocks=False))
f.write(os.linesep)
f.write(GetRow(30,'xii','More elastic labor supply'))
f.write(os.linesep)
f.write('% Note: with flexible prices, $x_t$ is pinned down by the firm\'s first order condition and we find $|dx/db| < 3\times10^5$ across the state space.')

                     
f.close()

In [ ]:
fname = os.path.join(OutDir,'MacroStabTableExtended.tex')
f = open(fname, "w")

f.write(os.linesep)
f.write(GetRow(1,'i','No aggr. shocks',Shocks=False))
f.write(os.linesep)
f.write(GetRow(1,'ii','Baseline'))
f.write(os.linesep)
f.write(GetRowFlex(8,'iii','Flexible prices'))
f.write(os.linesep)
f.write(GetRow(3,'iv','Aggressive monetary policy'))
f.write(os.linesep)
f.write(GetRow(12,'v','No $Q^u$'))
f.write(os.linesep)
f.write(GetRow(23,'vi','Smaller prod. shock'))
f.write(os.linesep)
f.write(GetRow(22,'vii','Smaller mon. shock'))
f.write(os.linesep)
f.write(GetRow(24,'viii','Smaller gov. shock'))
f.write(os.linesep)
f.write(GetRow(25,'ix','Less pers. prod. shock'))
f.write(os.linesep)
f.write(GetRow(26,'x','Less pers. mon. shock'))
f.write(os.linesep)
f.write(GetRow(27,'xi','Less pers. gov. shock'))
f.write(os.linesep)
f.write(GetRow(28,'xii','Lower elas. of wage to prod.'))
f.write(os.linesep)
f.write('% Note: with flexible prices, $x_t$ is pinned down by the firm\'s first order condition and we find $|dx/db| < 3\times10^5$ across the state space.')

                     
f.close()